In [5]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
import scipy.io
import numpy as np
from features import get_questions_matrix_sum, get_images_matrix, get_answers_matrix
from utils import grouper, selectFrequentAnswers

In [6]:
## DEFINE CONSTANTS
img_dim = 4096 #top layer of the VGG net
word_vec_dim = 300 #dimension of pre-trained word vectors
nb_hidden_units = 1024 #number of hidden units, a hyperparameter
nb_hidden_layers = 3
dropout = .5
activation = 'tanh'
num_epochs = 100
model_save_interval = 10
batch_size = 64

In [7]:
## OPEN TRAIN DATA
questions_train = open('../data/preprocessed/questions_train2014.txt', 'r').read().decode('utf8').splitlines()
answers_train = open('../data/preprocessed/answers_train2014_modal.txt', 'r').read().decode('utf8').splitlines()
images_train = open('../data/preprocessed/images_train2014.txt', 'r').read().decode('utf8').splitlines()

IOError: [Errno 2] No such file or directory: '../data/preprocessed/questions_train2014.txt'

In [ ]:
## ENCODE ANSWERS
labelencoder = preprocessing.LabelEncoder()
labelencoder.fit(answers_train)
nb_classes = len(list(labelencoder.classes_))

In [ ]:
## LOAD VGG FEATURES
vgg_model_path = '../features/coco/vgg_feats.mat'
features_struct = scipy.io.loadmat(vgg_model_path)
VGGfeatures = features_struct['feats']

In [ ]:
## LOAD word2vec
nlp = English()

In [ ]:
## DEFINE MODEL
model = Sequential()
model.add(Dense(nb_hidden_units, input_dim=img_dim+word_vec_dim, init='uniform'))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(nb_hidden_units, init='uniform'))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, init='uniform'))
model.add(Activation('softmax'))

In [ ]:
## COMPILE MODEL
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [ ]:
## TRAIN MODEL
for k in xrange(num_epochs):
    #shuffle the data points before going through them
    index_shuf = range(len(questions_train))
    shuffle(index_shuf)
    questions_train = [questions_train[i] for i in index_shuf]
    answers_train = [answers_train[i] for i in index_shuf]
    images_train = [images_train[i] for i in index_shuf]
    progbar = generic_utils.Progbar(len(questions_train))
    for qu_batch,an_batch,im_batch in zip(grouper(questions_train, batch_size, fillvalue=questions_train[-1]), 
                                          grouper(answers_train, batch_size, fillvalue=answers_train[-1]), 
                                          grouper(images_train, batch_size, fillvalue=images_train[-1])):
        X_q_batch = get_questions_matrix_sum(qu_batch, nlp)
        X_i_batch = get_images_matrix(im_batch, id_map, VGGfeatures)
        X_batch = np.hstack((X_q_batch, X_i_batch))
        Y_batch = get_answers_matrix(an_batch, labelencoder)
        loss = model.train_on_batch(X_batch, Y_batch)
        progbar.add(batch_size, values=[("train loss", loss)])
        #print type(loss)
    if k%model_save_interval == 0:
        model.save_weights(model_file_name + '_epoch_{:02d}.hdf5'.format(k))
model.save_weights(model_file_name + '_epoch_{:02d}.hdf5'.format(k))